In [44]:
import argparse
import torch
import torch.nn as nn
from sklearn.metrics import roc_auc_score, average_precision_score
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.nn import RGCNConv
from torch_geometric.loader import ClusterData, ClusterLoader
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm, trange
import gc
import warnings
warnings.filterwarnings('ignore')

device = 'cpu' # 'cuda' if torch.cuda.is_available() else 'cpu'

gc.collect()

3175

In [45]:
hetero_data = torch.load('../data/model/hetero_graph_data.pt')

In [26]:
hetero_data

HeteroData(
  customer={
    x=[161086, 12],
    index=[161086],
  },
  product={
    x=[2708, 15],
    index=[2708],
  },
  (customer, order, product)={ edge_index=[2, 11892915] },
  (product, class, product)={ edge_index=[2, 1796620] },
  (product, rev_class, product)={ edge_index=[2, 1796620] },
  (product, rev_order, customer)={ edge_index=[2, 11892915] }
)

In [46]:
train_data, val_data, test_data = T.RandomLinkSplit(
        num_val=0.2,
        num_test=0.2,
        is_undirected=True,
        add_negative_train_samples=True,
        disjoint_train_ratio=0,
        edge_types=[('customer', 'order', 'product'),
                    ('product', 'class', 'product')],
        rev_edge_types=[('product', 'rev_order', 'customer'), 
                        ('product', 'rev_class', 'product')]
    )(hetero_data.to_homogeneous())
# 产品节点特征本来是12，被0填充到15了，与客户节点对齐

train_loader = ClusterLoader(ClusterData(train_data, num_parts=128), batch_size=32)  # 训练集太大，分批量

Computing METIS partitioning...
Done!


In [47]:
def negative_sample(data):
    # 从训练集中采样与正边相同数量的负边
    neg_edge_index = negative_sampling(
        edge_index=data.edge_index, num_nodes=data.num_nodes,
        num_neg_samples=data.edge_label_index.size(1), method='sparse')
    # print(neg_edge_index.size(1))   # 3642条负边，即每次采样与训练集中正边数量一致的负边
    edge_label_index = torch.cat(
        [data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        data.edge_label,
        data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    return edge_label, edge_label_index

In [48]:
# 定义参数
node_types = train_data.node_type.unique().cpu().tolist()  # 变同质图后，被标记为0（客户）和1（商品）
num_relations = len(hetero_data.edge_types)
init_sizes = [train_data.x[train_data.node_type==1].shape[-1] for i in node_types]  # 变同质图之后其实都是一样的
# init_x = [hetero_data[node_type].x.to(device) for node_type in node_types]
in_feats = 16  # 直接指定即可，RGCN中有一个线性层转化输出统一到此维度
hidden_feats = 32  # 隐藏层，直接指定即可
out_channels = 16  # 输出层，直接指定，最终输出的解码器的输入维度是 2*out_channels

In [75]:
class RGCN_LP(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(RGCN_LP, self).__init__()
        self.conv1 = RGCNConv(in_channels, hidden_channels,
                              num_relations=num_relations)
        self.conv2 = RGCNConv(hidden_channels, out_channels,
                              num_relations=num_relations)
        self.lins = torch.nn.ModuleList()
        for i in range(len(node_types)):
            lin = nn.Linear(init_sizes[i], in_channels)
            self.lins.append(lin)

        self.fc = nn.Sequential(
            nn.Linear(2 * out_channels, 1),
            nn.Sigmoid()
        )

    def trans_dimensions(self, xs):
        res = []
        for x, lin in zip(xs, self.lins):
            res.append(lin(x))
        return torch.cat(res, dim=0)

    def encode(self, data):
        x = [data.x[data.node_type == node_type] for node_type in node_types]
        x = self.trans_dimensions(x)
        edge_index, edge_type = data.edge_index, data.edge_type
        x = self.conv1(x, edge_index, edge_type)
        x = F.relu(x)
        # x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_type)
        return x

    def decode(self, z, edge_label_index):
        # print(z.shape)
        src = z[edge_label_index[0]]  # ! edge_label_index中的索引值大于节点数了，需要检查，而且节点的索引应该是外层的 data.index 而不是这里的顺序
        dst = z[edge_label_index[1]]
        x = torch.cat([src, dst], dim=-1)
        x = self.fc(x)
        return x

    def forward(self, data):
        z = self.encode(data)
        z = self.decode(z, data.edge_label_index)
        return z

In [76]:
def get_metrics(out):
    auc = roc_auc_score(test_data.edge_label.cpu().numpy(), out[:, 1].cpu().numpy())
    ap = average_precision_score(test_data.edge_label.cpu().numpy(), out[:, 1].cpu().numpy())
    return auc, ap

In [77]:
def train():
    model = RGCN_LP(in_feats, hidden_feats, out_channels).to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
    criterion = torch.nn.BCELoss().to(device)
    min_epochs = 10
    min_val_loss = np.Inf
    summary = {'val_loss': [], 'test_auc': [], 'test_ap': []}
    # epoch_count = 0
    model.train()  # 设定模型为可训练
    for epoch in trange(20):
        for train_batch in train_loader:
            train_batch.to(device)
            optimizer.zero_grad()
            # edge_label, edge_label_index = negative_sample(train_batch)
            out = model(train_batch).view(-1)
            loss = criterion(out, train_batch.edge_label)
            loss.backward()
            optimizer.step()
        # validation
        val_loss, test_auc, test_ap = test(model, val_data, test_data)
        summary['val_loss'].append(val_loss)
        summary['test_auc'].append(test_auc)
        summary['test_ap'].append(test_ap)
        if epoch_count + 1 > min_epochs and val_loss < min_val_loss:
            min_val_loss = val_loss

        print('epoch {:03d} train_loss {:.8f} val_loss {:.4f} test_auc {:.4f} test_ap {:.4f}'
              .format(epoch_count, loss.item(), val_loss, test_auc, test_ap))
        epoch_count += 1
    return pd.DataFrame(summary)

@torch.no_grad()
def test(model, val_data, test_data):
    model.eval()  # 设定模型不可训练
    # cal val loss
    criterion = torch.nn.BCELoss().to(device)
    out = model(val_data, val_data.edge_label_index).view(-1)
    val_loss = criterion(out, val_data.edge_label)
    # cal metrics
    out = model(test_data, test_data.edge_label_index).view(-1)
    model.train()

    auc, ap = get_metrics(out, test_data.edge_label)

    return val_loss, auc, ap

In [78]:
summary = train()

  0%|          | 0/20 [04:31<?, ?it/s]


IndexError: index 72811 is out of bounds for dimension 0 with size 39520